In [1]:
import findspark
findspark.init()

import pyspark
import random



In [2]:
from pyspark import SparkConf, SparkContext 
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating


In [3]:
# Before initiation， modify default value of sprk conf(executor memory driver memory , etc)
sc = SparkContext()


In [4]:
data = sc.textFile("D:/Anaconda3/train.txt")
data.count()

12403575

In [5]:
ratings = data.map(lambda l: l.split(' ')).map(lambda l: Rating(l[0], l[1], l[2]))
ratings.take(10)

[Rating(user=199808, product=248969, rating=90.0),
 Rating(user=199808, product=2663, rating=90.0),
 Rating(user=199808, product=28341, rating=90.0),
 Rating(user=199808, product=42563, rating=90.0),
 Rating(user=199808, product=59092, rating=90.0),
 Rating(user=199808, product=64052, rating=90.0),
 Rating(user=199808, product=69022, rating=90.0),
 Rating(user=199808, product=77710, rating=90.0),
 Rating(user=199808, product=79500, rating=90.0),
 Rating(user=199808, product=82317, rating=90.0)]

In [6]:
rank = 15
numIterations = 20
model = ALS.train(ratings, rank, numIterations)

In [7]:
# model.save(sc= sc,path="target/tmp/myCollaborativeFilter")

In [7]:
data_test = sc.textFile("D:/Anaconda3/test.txt")
data_test.count()

120000

In [8]:
ratings_test = data_test.map(lambda l: l.split(' ')).map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
ratings_test.take(10)
ratings_test.count()

120000

In [9]:
testdata = ratings_test.map(lambda p: (p[0], p[1]))
testdata.take(10)


[(199810, 208019),
 (199810, 74139),
 (199810, 9903),
 (199810, 242681),
 (199810, 18515),
 (199810, 105760),
 (199812, 276940),
 (199812, 142408),
 (199812, 130023),
 (199812, 29189)]

In [10]:
predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
predictions.count()

119974

In [11]:
ratesAndPreds = ratings_test.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
ratesAndPreds.take(10)

[((199813, 184173), (0.0, 52.959590916119375)),
 ((199814, 232332), (0.0, 87.70558097654228)),
 ((199815, 118841), (0.0, 34.04502134605689)),
 ((199816, 23616), (0.0, 56.55580089840702)),
 ((199818, 283698), (0.0, 34.40088700464496)),
 ((199819, 36793), (0.0, 94.22779566674552)),
 ((199819, 247549), (0.0, -60.7849861300207)),
 ((199823, 170625), (0.0, 67.76466104885257)),
 ((199823, 254625), (0.0, 15.354385060092135)),
 ((199826, 179298), (0.0, 121.11961734750045))]

In [12]:
test = ratesAndPreds.map(lambda k: [k[0][0],k[0][1],k[1][1]])
test.take(10)

[[199813, 184173, 52.959590916119375],
 [199814, 232332, 87.70558097654228],
 [199815, 118841, 34.04502134605689],
 [199816, 23616, 56.55580089840702],
 [199818, 283698, 34.40088700464496],
 [199819, 36793, 94.22779566674552],
 [199819, 247549, -60.7849861300207],
 [199823, 170625, 67.76466104885257],
 [199823, 254625, 15.354385060092135],
 [199826, 179298, 121.11961734750045]]

In [13]:

test_list = test.take(120000)

In [14]:
test_list

[[199813, 184173, 52.959590916119375],
 [199814, 232332, 87.70558097654228],
 [199815, 118841, 34.04502134605689],
 [199816, 23616, 56.55580089840702],
 [199818, 283698, 34.40088700464496],
 [199819, 36793, 94.22779566674552],
 [199819, 247549, -60.7849861300207],
 [199823, 170625, 67.76466104885257],
 [199823, 254625, 15.354385060092135],
 [199826, 179298, 121.11961734750045],
 [199827, 92599, 53.047947899918285],
 [199827, 184197, 81.90198869206716],
 [199839, 266143, 71.66021848683503],
 [199844, 138272, 76.00528448896574],
 [199845, 169945, 60.35714190944245],
 [199846, 214726, 88.27440688994926],
 [199861, 249701, 92.54315764599491],
 [199873, 8379, 85.19834993704157],
 [199880, 22640, 51.516799806438996],
 [199882, 109824, -8.234545668946666],
 [199895, 59891, 2.675336667132683],
 [199902, 249140, 27.562252295480754],
 [199921, 48891, 44.80956758873533],
 [199934, 170214, 50.63744697923335],
 [199944, 67182, -17.37811185433786],
 [199955, 267631, 8.54462854650643],
 [199958, 2021

In [15]:
#Save result
import numpy as np
np.savetxt('pred_1.txt',test_list,fmt=['%d','%d','%f'])

In [16]:
for each in test_list:
    if each[0]==199810:
        print(each)

[199810, 105760, 78.74294202799251]
[199810, 242681, 35.79229842129224]
[199810, 18515, 52.876718302673794]
[199810, 208019, 90.49611119802172]
[199810, 74139, 34.913251726450596]
[199810, 9903, 46.120318020540054]


In [17]:
len(test_list)

119974

In [18]:
A = ratings_test.map(lambda r: ((r[0], r[1]), r[2]))

In [19]:
A.take(10)

[((199810, 208019), 0.0),
 ((199810, 74139), 0.0),
 ((199810, 9903), 0.0),
 ((199810, 242681), 0.0),
 ((199810, 18515), 0.0),
 ((199810, 105760), 0.0),
 ((199812, 276940), 0.0),
 ((199812, 142408), 0.0),
 ((199812, 130023), 0.0),
 ((199812, 29189), 0.0)]

In [ ]:
for each in test_list :
    if each[0] == 199810:
        print(each)
    

In [11]:
predictions = model.predictAll(testdata)
predictions.take(10)

[Rating(user=245397, product=237636, rating=126.57795785420575),
 Rating(user=200223, product=21780, rating=58.165093099196326),
 Rating(user=206835, product=149988, rating=108.47398418256765),
 Rating(user=237823, product=149988, rating=77.54807772418101),
 Rating(user=238932, product=271092, rating=46.779914953787326),
 Rating(user=200725, product=271092, rating=56.34493060276219),
 Rating(user=240469, product=271092, rating=49.66957319779283),
 Rating(user=208454, product=271092, rating=90.63227942669482),
 Rating(user=232370, product=271092, rating=64.74273794195409),
 Rating(user=221690, product=271092, rating=61.245960572920616)]

In [1]:
# sc.stop()

In [57]:
pData = data.take(100)
pData = sc.parallelize(pData)

In [58]:

ratings = pData.map(lambda l: l.split(' '))
ratings.take(10)

[['199810', '208019', '0'],
 ['199810', '74139', '0'],
 ['199810', '9903', '0'],
 ['199810', '242681', '0'],
 ['199810', '18515', '0'],
 ['199810', '105760', '0'],
 ['199812', '276940', '0'],
 ['199812', '142408', '0'],
 ['199812', '130023', '0'],
 ['199812', '29189', '0']]

In [4]:
sc.stop()
# \
#     .map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

In [59]:
ratings = ratings.map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
ratings.take(10)

[Rating(user=199810, product=208019, rating=0.0),
 Rating(user=199810, product=74139, rating=0.0),
 Rating(user=199810, product=9903, rating=0.0),
 Rating(user=199810, product=242681, rating=0.0),
 Rating(user=199810, product=18515, rating=0.0),
 Rating(user=199810, product=105760, rating=0.0),
 Rating(user=199812, product=276940, rating=0.0),
 Rating(user=199812, product=142408, rating=0.0),
 Rating(user=199812, product=130023, rating=0.0),
 Rating(user=199812, product=29189, rating=0.0)]